# Test out pygcn using the default CORA dataset

In [1]:
import os
os.chdir('/home/jiageng/Documents/fhr/pygcn/pygcn')
import pandas as pd 

In [2]:
import time
import argparse
import torch
import torch.nn.functional as F
import torch.optim as optim

from pygcn.utils import load_data, accuracy
from pygcn.models import GCN

In [3]:
parser = argparse.ArgumentParser()
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='Disables CUDA training.')
parser.add_argument('--fastmode', action='store_true', default=False,
                    help='Validate during training pass.')
parser.add_argument('--seed', type=int, default=42, help='Random seed.')
parser.add_argument('--epochs', type=int, default=200,
                    help='Number of epochs to train.')
parser.add_argument('--lr', type=float, default=0.01,
                    help='Initial learning rate.')
parser.add_argument('--weight_decay', type=float, default=5e-4,
                    help='Weight decay (L2 loss on parameters).')
parser.add_argument('--hidden', type=int, default=16,
                    help='Number of hidden units.')
parser.add_argument('--dropout', type=float, default=0.5,
                    help='Dropout rate (1 - keep probability).')

args = parser.parse_args(['--fastmode'])
args.cuda = not args.no_cuda and torch.cuda.is_available()

In [4]:
adj, features, labels, idx_train, idx_val, idx_test = load_data()

Loading cora dataset...


/home/jiageng/Documents/fhr/.venv/lib/python3.12/site-packages/pygcn-0.1-py3.12.egg/pygcn/utils.py:80: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:644.)


In [11]:
# Replace sparse tensor adj with an identity matrix of the same shape
adj_rand = torch.eye(adj.size(0)).to_sparse()

In [12]:
model = GCN(nfeat=features.shape[1],
            nhid=args.hidden,
            nclass=labels.max().item() + 1,
            dropout=args.dropout)
optimizer = optim.Adam(model.parameters(),
                       lr=args.lr, weight_decay=args.weight_decay)

In [13]:
def train(epoch):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    output = model(features, adj_rand)
    loss_train = F.nll_loss(output[idx_train], labels[idx_train])
    acc_train = accuracy(output[idx_train], labels[idx_train])
    loss_train.backward()
    optimizer.step()

    if not args.fastmode:
        # Evaluate validation set performance separately,
        # deactivates dropout during validation run.
        model.eval()
        output = model(features, adj_rand)

    loss_val = F.nll_loss(output[idx_val], labels[idx_val])
    acc_val = accuracy(output[idx_val], labels[idx_val])
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train.item()),
          'acc_train: {:.4f}'.format(acc_train.item()),
          'loss_val: {:.4f}'.format(loss_val.item()),
          'acc_val: {:.4f}'.format(acc_val.item()),
          'time: {:.4f}s'.format(time.time() - t))


def test():
    model.eval()
    output = model(features, adj_rand)
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()))


In [14]:
# Train model
t_total = time.time()
for epoch in range(args.epochs):
    train(epoch)
print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

# Testing
test()


Epoch: 0001 loss_train: 1.9851 acc_train: 0.0929 loss_val: 1.9899 acc_val: 0.1100 time: 0.0409s
Epoch: 0002 loss_train: 1.9749 acc_train: 0.0929 loss_val: 1.9819 acc_val: 0.1067 time: 0.0325s
Epoch: 0003 loss_train: 1.9723 acc_train: 0.1000 loss_val: 1.9751 acc_val: 0.1000 time: 0.0332s
Epoch: 0004 loss_train: 1.9548 acc_train: 0.1071 loss_val: 1.9545 acc_val: 0.1133 time: 0.0281s
Epoch: 0005 loss_train: 1.9425 acc_train: 0.1500 loss_val: 1.9405 acc_val: 0.1033 time: 0.0285s
Epoch: 0006 loss_train: 1.9259 acc_train: 0.1214 loss_val: 1.9301 acc_val: 0.1433 time: 0.0251s
Epoch: 0007 loss_train: 1.9067 acc_train: 0.1786 loss_val: 1.9231 acc_val: 0.1067 time: 0.0184s
Epoch: 0008 loss_train: 1.9065 acc_train: 0.1714 loss_val: 1.9097 acc_val: 0.1333 time: 0.0151s
Epoch: 0009 loss_train: 1.8738 acc_train: 0.2357 loss_val: 1.8991 acc_val: 0.1833 time: 0.0208s
Epoch: 0010 loss_train: 1.8665 acc_train: 0.2429 loss_val: 1.8933 acc_val: 0.2200 time: 0.0176s
Epoch: 0011 loss_train: 1.8529 acc_train